<a href="https://colab.research.google.com/github/Redjo1258/DataRepo/blob/main/SatelliteImageryDownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.4 MB/s eta 0:00:00


In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='langit-bejana')

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.


  Args:
      image (ee.Image): A Sentinel-2 image.


  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')


  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11


  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

  #Define Region Of Interest (ROI)
  roi = ee.Geometry.BBox( 110.671093,-7.642112,110.907643,-7.481813)

  #Image data collection
  dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
    .filterBounds(roi)
)


  visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
  }


  citra = dataset.mean().clip(roi)


  m = geemap.Map()
  m.set_center(110.8188121,-7.559209, 12)
  m.add_layer(citra, visualization, 'RGB')
m

Map(bottom=546491.0, center=[-7.498302303676608, 110.79678048046414], controls=(WidgetControl(options=['positi…

In [ ]:
task = ee.batch.Export.image.toDrive(image=citra,
                                     region=roi,
                                     crs='EPSG:4326',
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     description='surakarta',
                                     folder='ADS4',
                                     maxPixels=1e9)
task.start()